In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution()

# import tensorflow as tf
# import tensorflow.contrib.eager as tfe

# tfe.enable_eager_execution()

from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
# tf.disable_v2_behavior()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split

%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
dataframe = pd.read_csv("doginfo.csv")
#dataframe.head()
print(dataframe)

      kindCd neuterYn sexCd  weight  noticeDays  age2  processState
0         푸들        N     F    7.46          10    12             0
1        믹스견        N     M    7.00          14     1             1
2        믹스견        U     M    4.50          11     2             0
3        진도견        N     M   10.00           8     1             0
4        믹스견        N     M    6.00           8     4             0
...      ...      ...   ...     ...         ...   ...           ...
22787    믹스견        N     M    1.00          10     0             1
22788    믹스견        N     M    1.00          10     0             1
22789    믹스견        N     M    1.00          10     0             1
22790     푸들        U     F    6.00          12     3             0
22791    믹스견        N     M    3.50          10     0             0

[22792 rows x 7 columns]


In [7]:
# # 특성 열을 만들고 배치 데이터를 변환하는 함수
# def demo(feature_column):
#   feature_layer = layers.DenseFeatures(feature_column)
#   print(feature_layer(example_batch).numpy())

In [14]:
feature_columns=[]
neuterYn = feature_column.categorical_column_with_vocabulary_list(
      'neuterYn', ['N', 'U', 'Y'])

neuterYn_one_hot = feature_column.indicator_column(neuterYn)
feature_columns.append(neuterYn_one_hot)
# demo(neuterYn_one_hot)

sexCd = feature_column.categorical_column_with_vocabulary_list(
      'sexCd', ['F', 'M', 'Q'])

sexCd_one_hot = feature_column.indicator_column(sexCd)
feature_columns.append(sexCd_one_hot)
# demo(sexCd_one_hot)

# kindCd = feature_column.categorical_column_with_vocabulary_list(
#       'kindCd', ['푸들', '믹스견', '말티즈'])

# kindCd_one_hot = feature_column.indicator_column(kindCd)
# feature_columns.append(kindCd_one_hot)

kindCd_hashed= feature_column.categorical_column_with_hash_bucket('kindCd', hash_bucket_size=1000)
feature_columns.append(kindCd_hashed)
# demo(feature_column.indicator_column(kindCd_hashed))

print(feature_columns.numpy())

#demo(neuterYn_one_hot)

AttributeError: 'list' object has no attribute 'numpy'

In [9]:
for header in ['weight', 'noticeDays', 'age2']:
    feature_columns.append(feature_column.numeric_column(header))

In [21]:
# 임베딩 열의 입력은 앞서 만든 범주형 열입니다.
# kindCd_embedding = feature_column.embedding_column(kindCd, dimension=114)
# demo(kindCd_embedding)


[[ 0.    1.    0.    0.   10.    0.    1.    0.    1.7 ]
 [ 3.    1.    0.    0.   10.    0.    1.    0.    6.5 ]
 [ 0.    0.    1.    0.   11.    0.    1.    0.    6.28]
 [ 0.    1.    0.    0.   10.    1.    0.    0.    1.  ]
 [ 2.    1.    0.    0.   10.    0.    1.    0.    4.  ]]


In [11]:
feature_layer=tf.keras.layers.DenseFeatures(feature_columns)

In [13]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

print(feature_layer())

In [15]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
456/456 [==============================] - 3s 7ms/step - loss: 0.6718 - acc: 0.5937 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/5
456/456 [==============================] - 2s 5ms/step - loss: 0.6626 - acc: 0.6063 - val_loss: 0.6583 - val_acc: 0.6046
Epoch 3/5
456/456 [==============================] - 1s 3ms/step - loss: 0.6595 - acc: 0.6104 - val_loss: 0.6599 - val_acc: 0.6046
Epoch 4/5
456/456 [==============================] - 1s 3ms/step - loss: 0.6574 - acc: 0.6135 - val_loss: 0.6562 - val_acc: 0.6106
Epoch 5/5
456/456 [==============================] - 1s 3ms/step - loss: 0.6562 - acc: 0.6140 - val_loss: 0.6548 - val_acc: 0.6139


In [16]:
loss, accuracy = model.evaluate(test_ds)
print("정확도", accuracy)

143/143 [==============================] - 0s 3ms/step - loss: 0.6525 - acc: 0.6140
정확도 0.61395043
